In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
%cd /content/drive/MyDrive/Causal_Inference_Model/

/content/drive/MyDrive/Causal_Inference_Model


In [13]:
import warnings
warnings.filterwarnings('ignore')

In [14]:
import pandas as pd
import sys, os

In [23]:
sys.path.append(os.path.abspath('scripts/'))

In [27]:
from causal_pipeline import EDAPipeline
from plots import Plot
from filehundle import LoadData
from plots import Plot

In [42]:
causal_pipeline = EDAPipeline()
plot = Plot()

AttributeError: ignored

Initialize Loaded Data

In [29]:
trip_df = pd.read_csv("data/nb.csv")

In [31]:
trip_df.info()
# df_trip

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 536020 entries, 0 to 536019
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Trip ID           536020 non-null  int64 
 1   Trip Origin       536020 non-null  object
 2   Trip Destination  536020 non-null  object
 3   Trip Start Time   534369 non-null  object
 4   Trip End Time     536019 non-null  object
dtypes: int64(1), object(4)
memory usage: 20.4+ MB


In [33]:
driver_df = pd.read_csv("data/driver_locations_during_request.csv")

In [34]:
driver_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1557740 entries, 0 to 1557739
Data columns (total 8 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   id             1557740 non-null  int64  
 1   order_id       1557740 non-null  int64  
 2   driver_id      1557740 non-null  int64  
 3   driver_action  1557740 non-null  object 
 4   lat            1557740 non-null  float64
 5   lng            1557740 non-null  float64
 6   created_at     0 non-null        float64
 7   updated_at     0 non-null        float64
dtypes: float64(4), int64(3), object(1)
memory usage: 95.1+ MB


Drop rows with null value for column: Trip Start Time

In [35]:
trip_df['Trip Start Time'].isna().sum()

1651

In [36]:
trip_df = trip_df[trip_df['Trip Start Time'].notna()]

In [37]:
trip_df.shape

(534369, 5)

Check if Trip Start Time is weekend or not

In [40]:
trip_df['is_weekend'] = trip_df['Trip Start Time'].apply(lambda x: causal_pipeline.isWeekend(x))

In [41]:
trip_df['is_weekend'].value_counts()

0    427182
1    107187
Name: is_weekend, dtype: int64

Weekends distribution

In [ ]:
plot_count(trip_df, 'is_weekend')